In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from gen.curriculum import get_cbc_grouped_questions, get_rubrics_by_sub_strand
from gen.utils import *

## Generate questions

In [2]:
grouped_questions = get_cbc_grouped_questions(
    strand_ids=[1], 
    question_count=2,
    bloom_skill_count=1,
    is_debug=True,
)


✅ Question breakdown written to output/question_breakdown.json. Total: 1


In [3]:
all_question_list = generate_llm_question_list(
    grouped_question_data=grouped_questions,
    is_debug=True,
)

# If there was a generation error
if not isinstance(all_question_list, list):
    print(all_question_list)


Laboratory Apparatus and Instruments =========
📝 Input token count (gpt-4o): 708
📦 Raw LLM output:
 content='[\n  {\n    "question": "In the school laboratory, Amina is using a Bunsen burner to heat a beaker of water. Describe how she should adjust the air hole of the Bunsen burner to achieve a blue flame and explain why this type of flame is preferred for heating.",\n    "expected_answer": "Amina should open the air hole of the Bunsen burner to allow more air to mix with the gas, producing a blue flame. This type of flame is preferred because it is hotter and provides more efficient heating."\n  },\n  {\n    "question": "During a science experiment, Brian notices that the readings on the thermometer are fluctuating. Analyze the possible reasons for this fluctuation and suggest how he can ensure more stable readings.",\n    "expected_answer": "The fluctuation could be due to the thermometer not being properly immersed in the liquid or external temperature changes. To ensure stable rea

In [4]:
exam_questions = get_db_question_objects(
    all_question_list=all_question_list,
    is_debug=True,
)


✅ Question list to output/question_list.json. Total: 2


## Simulate student answers

In [5]:
def convert_to_single_qa_list(data):
    result = []
    for index, item in enumerate(data):
        questions = item["questions"]
        expected_answers = item["expected_answers"]

        if isinstance(questions, str):
            questions = [questions]
        if isinstance(expected_answers, str):
            expected_answers = [expected_answers]

        if questions and expected_answers:
            result.append({
                "id": index + 1,
                "question": questions[0],
                "expected_answer": expected_answers[0]
            })

    return result

exam_output = convert_to_single_qa_list(exam_questions)
print(exam_output)

[{'id': 1, 'question': 'In the school laboratory, Amina is using a Bunsen burner to heat a beaker of water. Describe how she should adjust the air hole of the Bunsen burner to achieve a blue flame and explain why this type of flame is preferred for heating.', 'expected_answer': 'Amina should open the air hole of the Bunsen burner to allow more air to mix with the gas, producing a blue flame. This type of flame is preferred because it is hotter and provides more efficient heating.'}, {'id': 2, 'question': 'During a science experiment, Brian notices that the readings on the thermometer are fluctuating. Analyze the possible reasons for this fluctuation and suggest how he can ensure more stable readings.', 'expected_answer': 'The fluctuation could be due to the thermometer not being properly immersed in the liquid or external temperature changes. To ensure stable readings, Brian should ensure the thermometer is fully immersed and wait for the liquid to reach thermal equilibrium before taki

In [6]:
sample_students = [{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {
    'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71},]

In [7]:
parsed_answers = generate_llm_exam_answers_list(
    llm=OPENAI_LLM_4O,
    exam_data=exam_output,
    student_data=sample_students,
    is_debug=True,
)

📝 Input token count (gpt-4o): 715
📦 Raw LLM output:
 content='```json\n[\n  {\n    "id": 1,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "Amina should open the air hole a bit to get a blue flame. It\'s better for heating."\n      },\n      {\n        "question_id": "2",\n        "answer": "The thermometer might be moving or not in the liquid right. He should keep it still."\n      }\n    ]\n  },\n  {\n    "id": 2,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "Amina should open the air hole fully to mix more air with the gas, making a blue flame. This flame is hotter and heats things faster."\n      },\n      {\n        "question_id": "2",\n        "answer": "The readings might change because the thermometer is not fully in the liquid or the room temperature is changing. Brian should make sure the thermometer is fully immersed and wait until the temperature stops changing before reading."\n      }\n    ]\n  },\n  {\n    "id": 3

In [8]:
# # ==== DB MOCK ANSWERS
# # ====================
# exam = []
# EXAM_FILE = "data/exam.json"

# with open(EXAM_FILE, "r") as f:
#     exam = json.load(f)

# print(exam)


# students = []
# STUDENT_FILE = "data/classroom.json"

# with open(STUDENT_FILE, "r") as f:
#     students = json.load(f)

# print(students)


# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam,
#     student_data=students,
#     is_debug=True,
# )

## Grade answers

In [9]:
def get_answers_for_question(target_question_id, start_count):
    result = []
    count = start_count
    for entry in parsed_answers:
        for answer in entry["answers"]:
            if answer["question_id"] == str(target_question_id):
                result.append({
                    "answer_id": count,
                    "answer": answer["answer"],
                })
                count += 1
    return result, count


grouped_answers_data = []
count = 1

for idx, q in enumerate(exam_questions):
    item = {
        "question": q["description"],
        "expected_answer": q["expected_answer"],
        "rubrics": get_rubrics_by_sub_strand(
            sub_strand_name=q["sub_strand"],
            curriculum_file="data/cbc_data.json",
        )
    }
    student_answers, count = get_answers_for_question(idx + 1, count)
    item["student_answers"] = student_answers

    grouped_answers_data.append(item)

In [10]:
ANSWERS_LIST_OUTPUT_FILE = "output/answers_list.json"
with open(ANSWERS_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(grouped_answers_data, f, ensure_ascii=False, indent=4)
print(f"✅ Mocked Answers list written to {ANSWERS_LIST_OUTPUT_FILE}")

✅ Mocked Answers list written to output/answers_list.json


In [11]:
parsed_grades = generate_llm_answer_grades_list(
    llm=OPENAI_LLM_4O,
    grouped_answers_data=grouped_answers_data,
    is_debug=True,
)


In the school laboratory, Amina is using a Bunsen burner to heat a beaker of water. Describe how she should adjust the air hole of the Bunsen burner to achieve a blue flame and explain why this type of flame is preferred for heating. =========
📝 Input token count (gpt-4o): 575
📦 Raw LLM output:
 content='```json\n[{"answer_id": 1, "score": 2}, {"answer_id": 2, "score": 4}, {"answer_id": 3, "score": 3}, {"answer_id": 4, "score": 3}]\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 582, 'total_tokens': 640, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BWiULpLE0VeHcG0HEGBi8I8BD1xNV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9c2a1fa4-4b